In [619]:
import pandas as pd
import folium
from geopy.distance import geodesic


### Dataset
+ To download the dataset, go the link below and download the 'Trajectory Data' and rename it 'T-Drive_Taxi_Trajectory'
+ https://www.microsoft.com/en-us/research/publication/t-drive-trajectory-data-sample/

In [620]:
taxis_numbers = 100
#time_interval = "6H"  # Pandas uses uppercase 'H' for hours
DATASET_FOLDER = "T-Drive_Taxi_Trajectory/"

df_pd = pd.DataFrame()
# Loop through taxi files and concatenate data
for i in range(1, taxis_numbers + 1):
    df_taxi = pd.read_csv(
        f"{DATASET_FOLDER}/{i}.txt",
        header=None,
        names=['id', 'datetime', 'longitude', 'latitude']
    )
    df_pd = pd.concat([df_pd, df_taxi], ignore_index=True)

# Convert 'datetime' column to datetime format
df_pd['datetime'] = pd.to_datetime(df_pd['datetime'], format="%Y-%m-%d %H:%M:%S")

# Create a 'coordinates' column as tuples of (longitude, latitude)
df_pd['coordinates'] = list(zip(df_pd['longitude'], df_pd['latitude']))
# Sort by 'datetime'
df_pd = df_pd.sort_values(by='datetime')
df_pd = df_pd.set_index('datetime')

In [621]:
# Beijing PROJECT CRS CODE
ESPG_CODE = 32650

In [622]:
import geopandas as gpd
from shapely.geometry import Point

# Transform pandas coordinate to GeoPanda Points
gdf = gpd.GeoDataFrame(df_pd, geometry=gpd.points_from_xy(df_pd.longitude, df_pd.latitude))
gdf.geometry.set_crs(f'EPSG:{ESPG_CODE}', inplace=True)
gdf['geometry'] = gdf.geometry.to_crs(epsg=ESPG_CODE)
gdf

,id,longitude,latitude,coordinates,geometry
datetime,,,,,
2008-02-02 13:31:16,57,116.30212,39.90739,"(116.30212, 39.90739)",POINT (116.302 39.907)
2008-02-02 13:31:18,34,116.40227,39.85325,"(116.40227, 39.85325)",POINT (116.402 39.853)
2008-02-02 13:31:46,34,116.40228,39.85352,"(116.40228, 39.85352)",POINT (116.402 39.854)
2008-02-02 13:31:47,71,116.30153,40.03647,"(116.30153, 40.03647)",POINT (116.302 40.036)
2008-02-02 13:32:03,10,116.44457,39.92157,"(116.44457, 39.92157)",POINT (116.445 39.922)
...,...,...,...,...,...
2008-02-08 17:38:51,38,116.40156,39.98177,"(116.40156, 39.98177)",POINT (116.402 39.982)
2008-02-08 17:38:58,5,116.59029,39.83682,"(116.59029, 39.83682)",POINT (116.59 39.837)
2008-02-08 17:39:03,4,116.52252,39.92111,"(116.52252, 39.92111)",POINT (116.523 39.921)


In [ ]:
# Load Shapefile dataset
districts_points = gpd.read_file("dataset/points.shp")

districts_points.geometry.set_crs(f'EPSG:{ESPG_CODE}', inplace=True, allow_override=True)
districts_points['geometry'] = districts_points.geometry.to_crs(epsg=ESPG_CODE)

# Only keeping the points with more than 1000 occurrences
unique_points_count = districts_points["type"].value_counts()
common_points = unique_points_count[unique_points_count > 1000].index

districts_points = districts_points[districts_points["type"].isin(common_points)]
districts_points

,osm_id,timestamp,name,type,geometry
0,25248785,None,None,traffic_signals,POINT (116.389 39.906)
1,25248786,None,None,traffic_signals,POINT (116.393 39.906)
2,25248790,None,None,traffic_signals,POINT (116.39 39.899)
3,25248791,None,None,traffic_signals,POINT (116.394 39.899)
4,25585082,None,None,traffic_signals,POINT (116.386 39.899)
...,...,...,...,...,...
53581,-134871450,None,龙腾苑四区北门,bus_stop,POINT (116.336 40.074)
53582,-134871449,None,龙腾苑五区东门,bus_stop,POINT (116.333 40.074)
53583,-134871448,None,龙腾苑四区西门,bus_stop,POINT (116.333 40.071)
53584,-134871447,None,龙腾苑四区西门,bus_stop,POINT (116.334 40.072)


In [624]:
districts_points["type"].unique()

array(['traffic_signals', 'crossing', 'motorway_junctio', 'bus_stop',
       'restaurant', 'switch', 'stop', 'buffer_stop', 'level_crossing',
       'subway_entrance'], dtype=object)

In [625]:

# Join the Taxis position to the points (traffic_signals, stop, bus_stop...)
gdf_with_points = gdf.sjoin_nearest(districts_points, how="left", distance_col="distance")
gdf_with_points["index_right"] = gdf_with_points["index_right"].map(districts_points.index.to_series())
gdf_with_points["Point"] = gdf_with_points["index_right"].map(districts_points["geometry"])

gdf_with_points["distance_points"] = gdf_with_points.apply(
	lambda row: geodesic((row.geometry.y, row.geometry.x) , (row.Point.y, row.Point.x)).meters,
	axis=1
)
gdf_with_points.drop(columns=["index_right", "osm_id", "timestamp", "name", "distance"], inplace=True)
gdf_with_points.rename({"type": "point_type"}, axis=1, inplace=True)

gdf_with_points


,id,longitude,latitude,coordinates,geometry,point_type,Point,distance_points
datetime,,,,,,,,
2008-02-02 13:31:16,57,116.30212,39.90739,"(116.30212, 39.90739)",POINT (116.302 39.907),crossing,POINT (116.302 39.907),68.445961
2008-02-02 13:31:18,34,116.40227,39.85325,"(116.40227, 39.85325)",POINT (116.402 39.853),bus_stop,POINT (116.404 39.855),214.597350
2008-02-02 13:31:46,34,116.40228,39.85352,"(116.40228, 39.85352)",POINT (116.402 39.854),bus_stop,POINT (116.403 39.855),218.672646
2008-02-02 13:31:47,71,116.30153,40.03647,"(116.30153, 40.03647)",POINT (116.302 40.036),traffic_signals,POINT (116.303 40.037),167.587821
2008-02-02 13:32:03,10,116.44457,39.92157,"(116.44457, 39.92157)",POINT (116.445 39.922),traffic_signals,POINT (116.445 39.922),5.507006
...,...,...,...,...,...,...,...,...
2008-02-08 17:38:51,38,116.40156,39.98177,"(116.40156, 39.98177)",POINT (116.402 39.982),traffic_signals,POINT (116.401 39.982),13.611192
2008-02-08 17:38:58,5,116.59029,39.83682,"(116.59029, 39.83682)",POINT (116.59 39.837),bus_stop,POINT (116.589 39.838),167.403771
2008-02-08 17:39:03,4,116.52252,39.92111,"(116.52252, 39.92111)",POINT (116.523 39.921),traffic_signals,POINT (116.522 39.922),152.181232


In [ ]:
# Load the Road Shapefile
districts_roads = gpd.read_file("dataset/roads.shp")

districts_roads.geometry.set_crs(f'EPSG:{ESPG_CODE}', inplace=True, allow_override=True)
districts_roads['geometry'] = districts_roads.geometry.to_crs(epsg=ESPG_CODE)

districts_roads["type"].unique()

array(['secondary', 'trunk', 'tertiary', 'residential', 'service',
       'unclassified', 'secondary_link', 'primary', 'pedestrian',
       'footway', 'trunk_link', 'primary_link', 'motorway',
       'tertiary_link', 'cycleway', 'motorway_link', 'path', 'steps',
       'living_street', 'construction', 'track', 'proposed', 'services',
       'platform', 'raceway', 'road', 'disused', 'bridleway', 'elevator',
       'rest_area', 'corridor', 'scramble'], dtype=object)

In [627]:
# Join the Road shapefile with the previous dataframe that contain the points
gdf_with_roads_points = gpd.sjoin_nearest(gdf_with_points, districts_roads, how="left", distance_col="distance")

gdf_with_roads_points["index_right"] = gdf_with_roads_points["index_right"].map(districts_roads.index.to_series())
gdf_with_roads_points["Road"] = gdf_with_roads_points["index_right"].map(districts_roads["geometry"])


## Code to calculate the distance between the road and the taxis
## Don't know if it's useful or not

# gdf_with_roads_points["distance_roads (meters)"] = gdf_with_roads_points.apply(
#     # lambda row: print(row.Road),
# 	lambda row: geodesic((row.geometry.y, row.geometry.x) , (row.Road.y, row.Road.x)).meters,
# 	axis=1
# )
gdf_with_roads_points.drop(columns=["index_right", "osm_id", "distance"], inplace=True)
gdf_with_roads_points.rename({"type": "road_type"}, axis=1, inplace=True)
gdf_with_roads_points.sort_values("datetime")

KeyboardInterrupt: 

In [ ]:

# colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
#  'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
#  'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
#  'gray', 'black', 'lightgray']


# Visual Rendering
map = folium.Map(
    location=[39.9042, 116.4074],
    zoom_start=12,
    tiles="https://webst01.is.autonavi.com/appmaptile?style=7&x={x}&y={y}&z={z}",
    attr="高德地图 (Gaode)"
)

# Add offset to the latitude and longitude point
# Need to verify if this happened everywhere or only on the Folium Map that use AMAP coordinate
offset_latitude, offset_longitude = 0.00145, 0.006253

# Only rendering points that are next to a traffic_signals to check if traffic jam is correlated to traffic_lights
# But can also be test to other specific points
traffic_pos = gdf_with_roads_points[gdf_with_roads_points["point_type"] == "traffic_signals"].sort_values("datetime")[:2500]
roads_lines = traffic_pos.Road.apply(lambda geo: list((y + offset_latitude, x + offset_longitude) for (x, y) in geo.coords))

# LightBlue Icon correspond to the taxis
# Red Icon correspond to the traffic_signals
for (index, row) in traffic_pos.iterrows():
    folium.Marker([row.geometry.y + offset_latitude, row.geometry.x + offset_longitude], icon=folium.Icon(color="lightblue"), popup=f"Distance to Traffic Stop {row.distance_points}").add_to(map)
    folium.Marker([row.Point.y + offset_latitude, row.Point.x + offset_longitude], icon=folium.Icon(color="red"), popup=f"Traffic Stop").add_to(map)
    if row["name"] != None:
	    folium.PolyLine(roads_lines[index], color='purple', popup=f"{row["name"]}").add_to(map)
map.save("map.html")